# TEAM MEMBER
### RUI TANG(100776184)
### JUAN ZHANG(100777497)

# Import Libraries

In [1]:
import tensorboard
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()
from gensim.models import word2vec
import pandas as pd

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tqdm import tqdm

In [2]:
import sys
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

print(sys.version)
print(tf.__version__)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5771397497374100967
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6932588160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9882027273703668772
physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5"
]
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
2.4.1


# Load Target Word List

In [3]:
with open('data.csv') as f:
    words = f.read().split()
print(f"{words[:10]}..., \nlen: {len(words)}")


['foolishness', 'hath', 'wholesome', 'takest', 'feelings', 'anger', 'vaivaswata', 'matrix', 'kindled', 'convict']..., 
len: 8266


# Load Training Sets from Wikipedia
src: https://www.kaggle.com/mikeortman/wikipedia-sentences

In [4]:
wiki_sentences = word2vec.Text8Corpus('./training_data/wikisent2.txt')
wiki_sentences

## Train

In [5]:
if os.path.exists('trained_model/model'):
    model = word2vec.Word2Vec.load('trained_model/model')
else:
    try:
        os.mkdir('trained_model')
    finally:
        model = word2vec.Word2Vec(wiki_sentences, min_count=10, size=200, workers=6)
        model.save('trained_model/model')


### Validate with similar words

In [6]:
model.wv.most_similar('facebook')

[('MySpace', 0.752065896987915),
 ('Myspace', 0.7405902147293091),
 ('myspace', 0.7384539246559143),
 ('Bandcamp', 0.7017084956169128),
 ('SoundCloud', 0.6831356883049011),
 ('Facebook', 0.6733940243721008),
 ('Instagram', 0.6456301808357239),
 ('Soundcloud', 0.6213083267211914),
 ('VEVO', 0.6119403839111328),
 ('Tumblr', 0.6043229699134827)]

## Check dimensions

In [7]:
model.wv['love']

array([-1.2833694e-01,  1.0655611e+00, -1.2495052e+00,  3.1526837e+00,
        1.9671712e+00,  3.1243446e-01,  3.9368448e-01,  1.1758024e+00,
       -9.9737883e-01, -1.0408983e+00, -1.9402514e+00,  6.8285233e-01,
        6.2164766e-01,  4.1402329e-02,  1.1577287e+00,  2.9419437e-01,
       -1.1730505e+00, -1.5785629e+00, -1.6132689e+00, -1.3061663e+00,
        3.2493539e+00,  3.4155602e+00, -2.3833787e+00,  4.2085606e-01,
       -2.6949177e+00, -4.8249298e-01, -2.6014468e-01, -2.3313053e+00,
        3.1287897e-02, -7.6339233e-01,  3.7794924e-01,  1.9031297e+00,
       -1.6259744e+00, -4.6760038e-01, -6.3492852e-01, -2.4849632e+00,
       -1.6212566e-01, -1.5919902e+00, -1.5060451e+00, -1.1123823e-01,
        1.3022894e+00, -1.9446456e+00, -1.1681114e+00,  3.9815712e-01,
       -1.2598902e+00,  7.5294399e-01,  4.7624545e+00, -2.0775719e+00,
       -1.9785737e+00,  2.5980601e-01, -2.6307733e+00,  7.1457249e-01,
       -9.0182394e-01,  6.9972908e-01, -5.6870437e-01,  3.9544237e-01,
      

# Generate Data for Visualization

In [8]:
log_dir = 'tb_logs'
try:
    os.mkdir(log_dir)
except:
    pass
not_trained = []
vectors = []
meta = []
with open(os.path.join(log_dir, 'meta.tsv'), 'w') as out_m:
    for word in words:
        if word in model.wv:
            print(word, file=out_m)
            vectors.append(model.wv[word])
            meta.append(word)
        else:
            not_trained.append(word)
print(f'{len(not_trained)} words are not contained in dataset')

1931 words are not contained in dataset


In [9]:
from tensorboard.plugins import projector

In [10]:
def visualization(vectors, path):
    y = tf.Variable(vectors)
    checkpoint = tf.train.Checkpoint(embedding=y)
    checkpoint.save(os.path.join(path, "embedding.ckpt"))
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = 'meta.tsv'
    projector.visualize_embeddings(path, config)

In [11]:
visualization(vectors, log_dir)

UnknownError: Failed to rename: tb_logs\embedding.ckpt-1_temp/part-00000-of-00001.data-00000-of-00001 to: tb_logs\embedding.ckpt-1.data-00000-of-00001 : Access is denied.
; Input/output error [Op:MergeV2Checkpoints]

# Load TensorBoard

In [13]:
%load_ext tensorboard

In [14]:
%tensorboard --logdir=tb_logs/

Reusing TensorBoard on port 6006 (pid 20188), started 0:19:23 ago. (Use '!kill 20188' to kill it.)

## Or Use "tensorboard --logdir=tb_logs/" in commandline